In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
if (document.getElementById('appmode-busy')) {
    window.onbeforeunload = function() {return}
}

In [ ]:
try:
    import urllib.parse as urlparse
except ImportError:
    import urlparse
parsed_url = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)

In [ ]:
import aiida

home = None

if aiida.__version__ < "1.0.0":
    import datetime
    import os
    import shutil
    import subprocess

    import ipywidgets as ipw

    display(
        ipw.HTML(
            f"""<font size="4">Warning! The aiidalab-home application is incompatible
                     with AiiDA {aiida.__version__} that is currently installed on your system. To fix
                     the problem you should switch to the 'support/aiida-0.12.x' branch.
                     This can be done either manually or by clicking on the button 
                     below.</font>"""
        )
    )

    def update_home(_=None):
        home = os.path.expanduser("~")
        home_app_path = os.path.join(home, "apps", "home")
        home_app_new_path = (
            home_app_path + "-backup-" + datetime.datetime.now().isoformat()
        )
        try:
            os.chdir(home)  # Before removing app folder - move to the user's home.
            shutil.move(home_app_path, home_app_new_path)
        except:
            pass
        subprocess.call(
            [
                "git",
                "clone",
                "https://github.com/aiidalab/aiidalab-home.git",
                home_app_path,
            ]
        )
        os.chdir(home_app_path)
        subprocess.call(["git", "checkout", "support/aiida-0.12.x"])
        display(
            ipw.HTML(
                f"""<font size="4">Downgrade successful. The old home app was placed under {home_app_new_path} folder.
        Please, reload the page to see the downgraded home app.</font>"""
            )
        )

    update_btn = ipw.Button(
        description="Downgrade home app",
        layout={"width": "initial"},
        button_style="danger",
    )
    update_btn.on_click(update_home)
    display(update_btn)
else:
    from home.start_page import AiidaLabHome

    home = AiidaLabHome()
    if "move_up" in parsed_url:
        home.move_updown(parsed_url["move_up"][0], -1)
    elif "move_down" in parsed_url:
        home.move_updown(parsed_url["move_down"][0], +1)
    display(home.render())

In [ ]:
%%javascript
// in case an app was moved up or down
// I need to remove the corresponding variable from the url
// othervise if I update page again the same move will happen

var url = window.location.toString();
function getQueryVariable(variable)
{
       var query = window.location.search.substring(1);
       var vars = query.split("&");
       for (var i=0;i<vars.length;i++) {
               var pair = vars[i].split("=");
               if(pair[0] == variable){return pair[1];}
       }
       return(false);
}
if (name = getQueryVariable("move_up")){
    url = url.replace("move_up=".concat(name), "");
} else if (name = getQueryVariable("move_down")) {
    url = url.replace("move_down=".concat(name), "");
}
window.history.pushState("", "", url);